# Adaptive

In [ ]:
import holoviews as hv
from holoviews.streams import Stream, param
hv.notebook_extension('bokeh')

In [ ]:
import numpy as np
import learner
from time import sleep
from random import randint
from functools import partial
import importlib
importlib.reload(learner)

In [ ]:
def func(x, wait=True):
    x = np.asarray(x)
    a = 10
    if wait:
        sleep(randint(1, 3))
    return np.sin(x) + 0.0001/(0.0001 + x**2)

# Parallel

In [ ]:
import tornado
from distributed import Client

io = tornado.ioloop.IOLoop.current()

# Initialize the learner
learner1d = learner.Learner1D()
learner1d.add_point(-1, func(-1))
learner1d.add_point(1, func(1))

In [ ]:
async def dask_run(learner):
    async with Client(asynchronous=True) as client:
        await learner.run(func, client, learner1d ,goal=lambda learner1d: learner1d.loss() < 0.000001)

def plot(data):
        xy = [(k, v) for k, v in sorted(data.items()) if v is not None]
        if not xy:
            return hv.Scatter([])
        x, y  = np.array(xy, dtype=float).T
        return hv.Scatter((x, y))

In [ ]:
data_stream = Stream.define('data', data=param.ObjectSelector(default=dict()))
dm = hv.DynamicMap(plot, streams=[data_stream()])
dm

In [ ]:
pc = tornado.ioloop.PeriodicCallback(lambda: dm.event(data=learner1d.data), 100)
pc.start()
io.add_callback(dask_run, learner)